In [3]:
import os
import shutil
import pandas as pd
from collections import defaultdict
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

In [4]:
# GPU 장치 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# 이미지 파일명에서 성별 및 스타일을 추출하는 함수 (뒤의 W/M으로 성별 구분)
def extract_info_from_filename(filename):
    # 파일명 예시: "W_00237_60_popart_W.jpg"
    parts = filename.split('_')
    if len(parts) < 4:
        return None, None  # 형식이 맞지 않는 파일명은 무시
    style = parts[3]  # 스타일 정보는 네 번째 요소
    gender = '여성' if parts[-1].startswith('W') else '남성'  # 파일명의 마지막 부분이 성별을 나타냄
    return gender, style

In [6]:
# 디렉토리 내 파일명으로 통계 정보를 추출하는 함수
def generate_statistics(directory):
    # 성별 & 스타일별 이미지 수를 저장할 딕셔너리
    stats = defaultdict(lambda: defaultdict(int))

    # 디렉토리 내 모든 파일명에 대해 성별과 스타일 정보 추출
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            gender, style = extract_info_from_filename(filename)
            if gender and style:
                stats[gender][style] += 1

    # 통계 정보를 DataFrame으로 변환
    stats_list = []
    for gender, style_dict in stats.items():
        for style, count in style_dict.items():
            stats_list.append([gender, style, count])

    stats_df = pd.DataFrame(stats_list, columns=['성별', '스타일', '이미지 수'])
    return stats_df

In [7]:
# Training 및 Validation 데이터 경로 (경로는 실제 데이터셋 위치로 변경해야 합니다)
training_image_dir = '/home/gyuha_lee/DCC2024/dataset/training_image'
validation_image_dir = '/home/gyuha_lee/DCC2024/dataset/validation_image'

In [8]:
# Training 데이터 통계
training_stats_df = generate_statistics(training_image_dir)

# Validation 데이터 통계
validation_stats_df = generate_statistics(validation_image_dir)

In [9]:
# Training 데이터 통계표 출력
print("Training 데이터 통계")
print(f"{'성별':<4} {'스타일':<15} {'이미지 수':>10}")
print("-" * 34)
for index, row in training_stats_df.iterrows():
    print(f"{row['성별']:<4} {row['스타일']:<15} {row['이미지 수']:>10}")

Training 데이터 통계
성별   스타일                  이미지 수
----------------------------------
남성   metrosexual            278
남성   ivy                    237
남성   sportivecasual         298
남성   mods                   269
남성   bold                   268
남성   hiphop                 274
남성   normcore               364
남성   hippie                 260
여성   kitsch                  91
여성   lounge                  45
여성   disco                   37
여성   bodyconscious           95
여성   sportivecasual         157
여성   hippie                  91
여성   normcore               153
여성   athleisure              67
여성   hiphop                  48
여성   lingerie                55
여성   oriental                78
여성   minimal                139
여성   feminine               154
여성   cityglam                67
여성   classic                 77
여성   punk                    65
여성   genderless              77
여성   powersuit              120
여성   grunge                  31
여성   ecology                 64
여성   space           

In [10]:
# Validation 데이터 통계표 출력
print("\nValidation 데이터 통계")
print(f"{'성별':<4} {'스타일':<15} {'이미지 수':>10}")
print("-" * 34)
for index, row in validation_stats_df.iterrows():
    print(f"{row['성별']:<4} {row['스타일']:<15} {row['이미지 수']:>10}")


Validation 데이터 통계
성별   스타일                  이미지 수
----------------------------------
남성   metrosexual             58
남성   normcore                51
남성   hippie                  82
남성   bold                    57
남성   ivy                     79
남성   mods                    80
남성   sportivecasual          52
남성   hiphop                  66
여성   minimal                 35
여성   powersuit               34
여성   feminine                44
여성   genderless              12
여성   disco                   10
여성   hippie                  14
여성   sportivecasual          48
여성   athleisure              14
여성   normcore                20
여성   bodyconscious           23
여성   cityglam                18
여성   kitsch                  22
여성   classic                 22
여성   oriental                18
여성   punk                    12
여성   ecology                 17
여성   military                 9
여성   lounge                   8
여성   grunge                  10
여성   hiphop                   8
여성   popart       

In [11]:
# 각각의 통계 정보를 CSV 파일로 저장
training_stats_df.to_csv("training_image_statistics.csv", index=False, encoding='utf-8-sig')
validation_stats_df.to_csv("validation_image_statistics.csv", index=False, encoding='utf-8-sig')

In [12]:
# DeepLabV3 모델 로드 (배경 제거용) 및 GPU로 전송
deeplab_model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval().to(device)

/home/gyuha_lee/miniconda3/envs/python311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gyuha_lee/miniconda3/envs/python311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# 배경 제거 함수
def remove_background(image):
    preprocess_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 이미지를 텐서로 변환하고 GPU로 전송
    input_tensor = preprocess_transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        # GPU에서 모델을 실행하여 배경 제거
        output = deeplab_model(input_tensor)['out'][0]
    
    # 결과를 CPU로 전환하여 마스크 생성
    output_predictions = output.argmax(0).byte().cpu().numpy()
    mask = output_predictions == 15  # DeepLabV3에서 사람 클래스는 15번 클래스
    
    # 이미지를 numpy 배열로 변환
    image_np = np.array(image)
    
    # 마스크를 이용해 배경 제거
    background_removed_image = np.zeros_like(image_np)
    background_removed_image[mask] = image_np[mask]
    
    return Image.fromarray(background_removed_image)


In [14]:
# 원본 이미지 폴더 및 배경 제거된 이미지를 저장할 폴더 경로
input_folder = '/home/gyuha_lee/DCC2024/dataset/training_image'  # 원본 이미지 폴더 경로
output_folder = '/home/gyuha_lee/DCC2024/dataset/training_image_bg_removed'  # 배경 제거된 이미지 저장 폴더

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

In [15]:
# 폴더 내 모든 이미지에 대해 배경 제거 및 저장
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 이미지 파일만 처리
        # 이미지 열기
        img_path = os.path.join(input_folder, filename)
        image = Image.open(img_path).convert('RGB')
        
        # 배경 제거
        result_image = remove_background(image)
        
        # 배경 제거된 이미지 저장 경로
        save_path = os.path.join(output_folder, filename)
        
        # 배경 제거된 이미지 저장
        result_image.save(save_path)

        print(f"배경 제거 완료 및 저장: {save_path}")

print("모든 이미지의 배경 제거 완료 및 저장이 완료되었습니다.")

배경 제거 완료 및 저장: /home/gyuha_lee/DCC2024/dataset/training_image_bg_removed/W_01509_00_metrosexual_M.jpg


KeyboardInterrupt: 

In [12]:
# Custom Dataset 클래스 정의 (배경 제거 포함)
class CustomFashionDatasetWithBGRemoval(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
        self.classes = self.get_classes()
        
    def get_classes(self):
        classes = set()
        for filename in self.image_files:
            gender, style = extract_info_from_filename(filename)
            classes.add((gender, style))
        return sorted(list(classes))
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        image = Image.open(img_path).convert("RGB")
        
        # 배경 제거 적용
        image = remove_background(image)
        
        if self.transform:
            image = self.transform(image)
        
        gender, style = extract_info_from_filename(img_name)
        label = self.classes.index((gender, style))
        
        return image, label


In [13]:
# 파일명에서 성별 및 스타일 정보를 추출하는 함수 (뒤의 W/M으로 성별 구분)
def extract_info_from_filename(filename):
    parts = filename.split('_')
    style = parts[3]
    gender = '여성' if parts[-1].startswith('W') else '남성'
    return gender, style

In [14]:
# 데이터 전처리 정의
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [15]:
train_dataset = CustomFashionDatasetWithBGRemoval(training_image_dir, transform=transform_train)
val_dataset = CustomFashionDatasetWithBGRemoval(validation_image_dir, transform=transform_val)

In [21]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

In [22]:
# ResNet-18 모델 정의
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [23]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [24]:
# 모델 학습 및 검증 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        
        # 학습 단계
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # Backward + Optimizer
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        
        # 검증 단계
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

In [ ]:
# 모델 학습 및 검증
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25)

In [ ]:
# 학습된 모델 저장
torch.save(model.state_dict(), 'resnet18_with_bg_removal.pth')